# IPM Departamento

ivan corredor 22-02-2024

In [1]:
import pandas as pd
import numpy as np

## Cargue de datos y estandarización inicial

1. Cargar base de víctimas para cada año.
2. homogenizar los nombres de las variables de la base de víctimas 2022, 
para que sean iguales a los de los 3 años anteriores,
dejando los nombres variables de todas las bases en mayúsculas.

In [2]:
## ruta entrada
p1 = "C:/Users/ivan.corredor/pobreza_calculos_2019_2022/2023/multidimensional/"
## ruta salida
p2 = "C:/Users/ivan.corredor/pobreza_calculos_2019_2022/2023/multidimensional/salidas/"

In [3]:
# cargar datos y pasar a mayusculas
# las bases vienen separadas como se describe a continuación;
# victimas 2022: (,) # personas 2021,2022: (,) ## hogares 2021,2022: (,) ## vivienda 2021,2022: (,)
# para los demás años las bases están separadas por (;)

ext1 = [";", ";", ";", ","]
ext2 = [";", ";", ",", ","]

for i, j, k in zip(range(19,23), ext1, ext2):
    #víctimas
    df1 = pd.read_csv(p1 + f"20{i}/factores_victima_20{i}.csv", sep= j, decimal= ",")
    df1.columns= df1.columns.str.upper()
    exec(f"ipm_{i}_v = df1.copy()")

    # personas
    df2 = pd.read_csv(p1 + f"20{i}/personas_dpto_20{i}.csv", sep= k, decimal= ",")
    df2.columns= df2.columns.str.upper()
    df2.rename(columns = {"P6020":"SEXO", "P6040":"EDAD"}, inplace = True)
    exec(f"ipm_{i}_p = df2.copy()")

    #hogares
    df3 = pd.read_csv(p1 + f"20{i}/hogares_dpto_20{i}.csv", sep= k, decimal= ",")
    df3.columns= df3.columns.str.upper()
    exec(f"ipm_{i}_h = df3.copy()")

    #vivienda
    df4 = pd.read_csv(p1 + f"20{i}/viviendas_dpto_20{i}.csv", sep= k, decimal= ",")
    df4.columns= df4.columns.str.upper()
    exec(f"ipm_{i}_viv = df4.copy()")
    

In [4]:
# renombrar variables solo de 2022
ipm_22_v.rename(columns = {"VICTIMA":"ES_VICTIMA", "DESPLAZADO":"ES_DESPLAZADO", "FEX_VIC":"FEX_C_VICTI"}, inplace = True)

## Pegue de bases

### Cruce personas - víctimas. 2019 - 2022

In [5]:
#etapa 1
#tablas
t_per = [ipm_19_p, ipm_20_p, ipm_21_p, ipm_22_p]
t_vic = [ipm_19_v, ipm_20_v, ipm_21_v, ipm_22_v]

for i,j,k in zip (range(19,23), t_per, t_vic):
    #merge
    df_pv =  pd.merge(left= j, right = k,
                        left_on=['DIRECTORIO',  'SECUENCIA_P', 'ORDEN'], 
                        right_on=['DIRECTORIO', 'SECUENCIA_P','ORDEN'],
                        how='left',
                        sort=False, validate='1:1', suffixes=('_P', '_VIC'), 
                        indicator = 'INDICATOR_COLUMN')
    #change format variables
    names = ['ES_VICTIMA', 'ES_DESPLAZADO','FEX_C_VICTI']
    for l in names:
        df_pv[l] = pd.to_numeric(df_pv[l], errors='coerce')
    # recalculate dummies
    df_pv = df_pv.assign(VIC2 = df_pv.groupby(['DIRECTORIO', 
                                               'SECUENCIA_P'])['ES_VICTIMA'].transform(lambda x: 1 if (x.sum() >= 1) else 0),
                         DESPLA2 = df_pv.groupby(['DIRECTORIO',
                                                  'SECUENCIA_P'])['ES_DESPLAZADO'].transform(lambda x: 1 if (x.sum() >= 1) else 0),
                         FEXHOGVIC = df_pv.groupby(['DIRECTORIO',
                                                    'SECUENCIA_P'])['FEX_C_VICTI'].transform('mean')
                        )
    #create objets with info per-vic
    exec(f"per_vic_{i} = df_pv.copy()")

### Cruce personas, víctimas - hogares. 2019-2022

In [6]:
# etapa 2
# tablas de la etapa 1
# hogares
t_hog = [ipm_19_h, ipm_20_h, ipm_21_h, ipm_22_h] 
#per-vic
t_pv = [per_vic_19, per_vic_20, per_vic_21, per_vic_22]

for i, j, k in zip(range(19,23), t_hog, t_pv):
    # merge
    df_pvh = pd.merge(left = j, right = k, 
                               left_on=['DIRECTORIO', 'SECUENCIA_ENCUESTA'], 
                               right_on=['DIRECTORIO', 'SECUENCIA_P'],
                               how='left', 
                               sort=False, validate='1:m', suffixes=("_H", "_PV"), 
                               indicator = 'INDICATOR_COLUMN2')
    #create objets with info per-vic-hog
    exec(f"per_vic_hog_{i} = df_pvh.copy()")

In [7]:
# etapa 3
# tablas de la etapa 2
#per-vic-hog
t_pvh = [per_vic_hog_19, per_vic_hog_20, per_vic_hog_21, per_vic_hog_22]
#viviendas
t_viv = [ipm_19_viv, ipm_20_viv, ipm_21_viv, ipm_22_viv] 

for i, j, k in zip(range(19,23), t_pvh, t_viv):
    # merge
    df_pvh_viv = pd.merge(left = j, right = k, 
                               left_on=['DIRECTORIO'], 
                               right_on=['DIRECTORIO'],
                               how='inner', 
                               sort=False, validate='m:1', suffixes=("_PVH", "_VIV"), 
                               indicator = 'INDICATOR_COLUMN3')
    #create objets with info per-vic-hog
    exec(f"per_vic_hog_viv{i} = df_pvh_viv.copy()")

## Agregar variables

In [8]:
##crear variable no víctima
def novic(df):
    if df["VIC2"]!=1:
        x = 1
    else:
        x= 0
    return x

In [9]:
# rango de edad
def rang_ed(df):
    if df["EDAD"]<=5:
        x = "Entre 0 y 5 años"
    elif (df["EDAD"]>5 and df["EDAD"]<=11):
        x = "Entre 6 y 11 años"
    elif (df["EDAD"]>11 and df["EDAD"]<=17):
        x = "Entre 12 y 17 años"    
    elif (df["EDAD"]>17 and df["EDAD"]<=28):
        x = "Entre 18 y 28 años"
    elif (df["EDAD"]>28 and df["EDAD"]<=59):
        x = "Entre 29 y 59 años"
    else:
        x = "60 años o más"
    return x  

In [10]:
# tablas de salida pora cada año 2019-2022
t_finals = [per_vic_hog_viv19, per_vic_hog_viv20, per_vic_hog_viv21, per_vic_hog_viv22]
#variables para cambio de formato
names_var = [ "VIC2", 'DESPLA2','POBRE',"FEXHOGVIC", "NO_VICTIMA", "FEX_C_H", "P3"]

for i, j in zip(t_finals, range(2019,2023)):

    i["PERIODO"] = j
    i["NO_VICTIMA"] = i.apply(novic, axis = 1)
    i["GRUPO_ETARIO"] = i.apply(rang_ed, axis = 1)
    i["NOMBRE_REGION"] = i["REGION"].replace([1,2,3,4,5,6,7,8,9], 
                                                           ["Caribe", "Oriental", "Central", 
                                                             "Pacífica (sin valle)", "Bogotá", 
                                                             "Antioquia", "Valle del Cauca", 
                                                             "San Andrés y Providencia", "Orinoquía y Amazonía"])
    i["NOMBRE_DEPARTAMENTO"] = i["DEPARTAMENTO"].replace([5,8,11,13,15,17,18,19,20,23,25,27,41,44,47,50,52,54,63,66,68,
                                                                70, 73, 76, 81, 85,86, 88, 91,94, 95,97,99],
                                                                ["Antioquia", "Atlántico", "Bogotá", "Bolívar","Boyacá", 
                                                                 "Caldas","Caquetá","Cauca", "Cesar","Córdoba", "Cundinamarca",
                                                                "Chocó", "Huila", "La Guajira", "Magdalena", "Meta", "Nariño",
                                                                "Norte de Santander", "Quindio", "Risaralda", "Santander",
                                                                "Sucre", "Tolima", "Valle del Cauca", "Arauca", "Casanare",
                                                                "Putumayo", "Providencia y Santa Catalina", "Amazonas", 
                                                                "Guainía", "Guaviare", "Vaupés", "Vichada"])
    ### colocar formato numerico a las variables a operar
    for k in names_var:
        i[k] = pd.to_numeric(i[k], errors='coerce')

In [11]:
# ajustes adionales a cada una de las bases finales por año
#renombrar variables
per_vic_hog_viv19.rename(columns= {'P6390S1':'P6390'}, inplace=True)

# borrar variable parentesco P6051 ya que no viene en la base de personas de 2021
# Esto es un error ya que se necesita esta variable para calcular la pobreza por sexo o rango de edad.
# Por ello se deja comentario
# En nueva revisión DANE manifiesta que se puede utilizar la variable ORDEN = 1 para jefe de hogar
# por ello se agrega está variable al año 2021.

per_vic_hog_viv21["ORDEN"] = pd.to_numeric(per_vic_hog_viv21["ORDEN"], errors='coerce')
per_vic_hog_viv21["P6051"] = per_vic_hog_viv21['ORDEN'].apply(lambda x: 1 if x == 1 else 0)

###BORRAR 4 VARIABLES de 2021 QUE NO SE ENCUENTRAN EN 2019 Y 2020
per_vic_hog_viv21.drop(columns =[ 'P3336S1', 'P3336S2','P3336S3', 'P3337'], inplace =True)

###BORRAR 4 VARIABLES de 2022 QUE NO SE ENCUENTRAN EN 2019 Y 2020
per_vic_hog_viv22.drop(columns =[ 'P3336S1', 'P3336S2','P3336S3', 'P3337'], inplace =True)

In [12]:
per_vic_hog_viv21[per_vic_hog_viv21["PERIODO"]==2021]["P6051"].value_counts()

P6051
0    168386
1     89203
Name: count, dtype: int64

In [13]:
pd.options.display.max_columns = None
per_vic_hog_viv22.head()

,PERIODO,DIRECTORIO,SECUENCIA_ENCUESTA_PVH,SECUENCIA_P_H,P5010,P8526,P8530,P1075,P1077S21,P1077S22,P1077S23,FEX_C_H,REGION,DEPARTAMENTO,PERSONAS,PAREDES,PISOS,ALCANTARILLADO,ACUEDUCTO,EMPLEO_FORMAL,DESEMPLEO_LARGA_DURACION,BARRERAS_ACCESO_SALUD,ASEGURAMIENTO_SALUD,TRABAJO_INFANTIL,ATENCION_INTEGRAL,INASISTENCIA_ESCOLAR,REZAGO_ESCOLAR,ALFABETISMO,LOGRO_EDUCATIVO,HACINAMIENTO,IPM,POBRE,FEXP,SECUENCIA_ENCUESTA_P,SECUENCIA_P_PV,ORDEN,P6051,SEXO,EDAD,FEX_C_PV,P6090,P5665,P8563,P51,P55,P774,P6160,P8586,P8587,P8587S1,P1088,P1088S1,P6180,P6240,P6250,P6260,P6270,P6351,P6390,P7250,P6920,P1082S2,SECUENCIA_ENCUESTA_VIC,ES_VICTIMA,ES_DESPLAZADO,FEX_C_VICTI,INDICATOR_COLUMN,VIC2,DESPLA2,FEXHOGVIC,INDICATOR_COLUMN2,SECUENCIA_ENCUESTA_VIV,SECUENCIA_P,P3,P4005,P4015,P8520S3,P8520S5,INDICATOR_COLUMN3,NO_VICTIMA,GRUPO_ETARIO,NOMBRE_REGION,NOMBRE_DEPARTAMENTO
0,2022,7787900,1,1,2,2,8,1,2,2,1,143.618606,1,13,4,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0.28,0,574.47442267,1,1,1,1,2,37,143.61860567,1,2,NaN,NaN,NaN,NaN,1.0,2.0,5.0,11.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,COMERCIO DE PRODUCTOS ALIMENTICIOS ASEO,NaN,2.0,1.0,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,5,6,2,1,both,1,Entre 29 y 59 años,Caribe,Bolívar
1,2022,7787900,1,1,2,2,8,1,2,2,1,143.618606,1,13,4,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0.28,0,574.47442267,2,1,2,3,2,13,143.61860567,1,2,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,3.0,8.0,2.0,3.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,5,6,2,1,both,1,Entre 12 y 17 años,Caribe,Bolívar
2,2022,7787900,1,1,2,2,8,1,2,2,1,143.618606,1,13,4,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0.28,0,574.47442267,3,1,3,3,2,5,143.61860567,1,2,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,5,6,2,1,both,1,Entre 0 y 5 años,Caribe,Bolívar
3,2022,7787900,1,1,2,2,8,1,2,2,1,143.618606,1,13,4,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0.28,0,574.47442267,4,1,4,5,1,73,143.61860567,1,2,NaN,NaN,NaN,NaN,1.0,2.0,3.0,5.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,SERVICIOS VENTA DE JUEGOS DE AZAR GIROS RECARGAS,NaN,2.0,1.0,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,5,6,2,1,both,1,60 años o más,Caribe,Bolívar
4,2022,7787901,1,1,2,1,1,1,2,2,2,273.731538,1,13,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.1,0,1094.9261524,1,1,1,1,1,55,273.73153811,1,2,NaN,NaN,NaN,NaN,1.0,2.0,5.0,11.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,COMPRA Y VENTA DE FRUTAS,NaN,2.0,1.0,NaN,NaN,NaN,NaN,left_only,0,0,NaN,both,1,1,1,1,4,1,1,both,1,Entre 29 y 59 años,Caribe,Bolívar


## Concatenar

In [14]:
dpto_19_22 = pd.concat(t_finals, axis=0)

## Exportar

In [15]:
#exportar
dpto_19_22.to_csv(p2+ "dpto_19_22.csv", sep=";", index =False)